In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

In [2]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [3]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [4]:
multiply.name

'multiply'

In [5]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [6]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [7]:
# tool binding

In [8]:
load_dotenv()
model = ChatOpenAI(model = "gpt-4.1-nano-2025-04-14", temperature=0.2, max_completion_tokens = 200)

In [9]:
model.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtKk1lrqHXYBOJbT4gYVMg0fGoHku', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6514d44c-c0c9-48c2-8cda-78157ff27e04-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
llm_with_tools = model.bind_tools([multiply])

In [11]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="Hello! I'm doing well, thank you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 57, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtKk4Y8X5gFC4FbNz2roVt7Gzbzf7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--407afb4c-65a5-4ecc-9b0f-84ab2575bb3c-0', usage_metadata={'input_tokens': 57, 'output_tokens': 17, 'total_tokens': 74, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
query = HumanMessage("can you multiply 3 with 100")

In [13]:
messages = [query]

In [14]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={})]

In [15]:
result = llm_with_tools.invoke(messages)

In [16]:
messages.append(result)

In [17]:
tool_result = multiply.invoke(result.tool_calls[0])

In [18]:
messages.append(tool_result)

In [19]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HEhxIKyhUBh9B5JwISAFHikj', 'function': {'arguments': '{"a":3,"b":100}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 61, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtKk6eX47CJPbc8fdfGt6neRr8grB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--45687c75-e33e-4b48-add7-d9ff1002d33b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'call_HEhxIKyhUBh9B5JwISAFHikj', 'type': 'tool_call'}], usage_metadata={'input_to

In [20]:
llm_with_tools.invoke(messages).content

'The result of multiplying 3 with 100 is 300.'

### Currency Conversion Tool

In [21]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/877899da8abd3396b7418bb9/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float: #LLM doesnot fill the argument but the developer will inject the argument at run time
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [22]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1752451201,
 'time_last_update_utc': 'Mon, 14 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1752537601,
 'time_next_update_utc': 'Tue, 15 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.9457}

In [23]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [24]:
convert.invoke({'base_currency_value': 10, 'conversion_rate': 85.9457})

859.457

In [25]:
llm = ChatOpenAI(model = 'gpt-4.1-nano-2025-04-14')

In [26]:
llm_with_tools1 = llm.bind_tools([get_conversion_factor, convert])

In [27]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [28]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [29]:
ai_messages = llm_with_tools1.invoke(messages)

In [30]:
messages.append(ai_messages)

In [31]:
ai_messages.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_zsEHZZyslb3feIyocoW821dT',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_oq4xGkZJD62amLiOutRn0o2h',
  'type': 'tool_call'}]

In [32]:
import json

for tool_call in ai_messages.tool_calls:
    #execute the 1st tool and get the value of conversion rate
    if tool_call['name'] ==  'get_conversion_factor':
        tool_meesage1 = get_conversion_factor.invoke(tool_call)
        #fetch the conversion rate
        conversion_rate = json.loads(tool_meesage1.content)['conversion_rate']
        #append the tool message to message list
        messages.append(tool_meesage1)
    #execute the 2nd tool using the conversion rate from tool 1
    if tool_call['name'] == 'convert':
        #fetch the current args
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [33]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zsEHZZyslb3feIyocoW821dT', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_oq4xGkZJD62amLiOutRn0o2h', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 119, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtKkHmHemKo6q95YzFxxXCH1hsh8L', 'service_tier': 

In [34]:
llm_with_tools1.invoke(messages).content

'The conversion factor between INR and USD is approximately 0.01164. Therefore, 10 INR is approximately 0.1164 USD.'

In [35]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\Pc\AppData\Local\Temp\ipykernel_17380\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [36]:
# --- Step 6: Run the Agent ---
user_query = "What is the product of 3 and 10"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: I need to calculate the product of 3 and 10. Since this is a simple multiplication, I can do it directly without needing a tool.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The product of 3 and 10 is 30."
}
```

> Finished chain.
